In [1]:
import torch
import pyro
from pyro.optim import Adam
import congas as cg
from congas.models import MixtureCategorical
from pyro.infer import TraceEnum_ELBO


In [2]:
data = cg.simulation_data

interface = cg.Interface()

In [3]:
data["norm_factor"] = torch.sum(data["data"],axis=0) / 10**3

In [4]:
torch.mean((data["data"] / data["norm_factor"]) / 2 )

tensor(15.6250)

In [5]:
interface.set_model(MixtureCategorical)
interface.set_optimizer(Adam)
interface.set_loss(TraceEnum_ELBO)
interface.initialize_model(data)
interface.set_model_params({"lr" : 0.05,'K': 2, "nb_size_init" : torch.ones(32) * 300,
                            "theta_scale" : torch.ones(32) * 0.15,
                            "theta_rate" : torch.ones(32) * 0.01, "binom_prior_limits" : [30, 10000])

In [6]:
ll = interface.run(steps = 600, MAP = True)



Running MixtureCategorical on 1000 cells wiht 32 segments for 600 steps

Done!


ELBO: 104246.520000000  : 100%|██████████| 600/600 [01:49<00:00,  5.47it/s]


In [7]:
import numpy as np

lr = interface.learned_parameters()
lr

Computing assignment probabilities


{'mixture_weights': array([0.5, 0.5], dtype=float32),
 'NB_size': array([1077.7509  , 5812.098   ,  583.19464 , 1894.6548  ,  432.82162 ,
         890.142   , 2328.7944  ,   30.453539,  768.9554  , 4156.346   ,
         816.6374  ,   30.477314, 1367.5786  , 2909.3667  ,  809.7705  ,
         373.9498  ,  946.7345  ,  367.3825  ,  892.391   , 1998.3137  ,
        2599.5688  , 2683.2703  , 5075.257   ,  389.80185 ,  266.25543 ,
        2905.0903  , 3977.9055  , 2178.833   ,  379.2625  ,  911.1395  ,
         142.75395 ,  406.97247 ], dtype=float32),
 'segment_factor': array([ 9.55804  , 13.469532 ,  8.338312 , 10.242219 ,  6.4349046,
         9.488064 , 22.83265  ,  8.476664 ,  9.4744425, 18.805866 ,
         8.902912 , 10.002401 ,  9.486863 , 11.829577 ,  9.774191 ,
         7.1009603, 10.284591 ,  7.13804  ,  9.502514 , 23.005589 ,
        11.819198 , 13.4221325, 12.627545 ,  6.407587 ,  6.3813405,
        11.8229885, 12.616474 , 10.308607 ,  7.1666355,  8.6883   ,
         4.203578 , 

In [8]:
import numpy as np
np.argmax(lr["assignment_probs"], axis = 0)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [9]:
print(np.argmax(lr["CNV_probabilities"][0], axis = 1) + 1)
print(np.argmax(lr["CNV_probabilities"][1], axis = 1) + 1)


[3 3 2 3 3 3 3 2 3 3 2 3 3 3 4 3 3 3 3 4 2 3 3 3 3 3 3 3 3 3 2 3]
[3 3 2 3 3 3 3 2 3 3 2 3 3 3 4 3 3 3 3 4 2 3 3 3 3 3 3 3 3 3 2 3]


In [10]:
data["pld"]

tensor([2.0000, 2.0000, 1.0000, 2.0000, 2.0000, 2.0000, 2.0000, 1.3000, 2.0000,
        2.0000, 1.0000, 1.7000, 2.0000, 2.0000, 3.0000, 2.0000, 2.0000, 2.0000,
        2.0000, 3.0000, 1.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000,
        2.0000, 2.0000, 2.0000, 1.0000, 2.0000])